# DataFrame2ProjectTable

Load a DataFrame of repositories and write it to a GitHub project table

In [1]:
import os
from dotenv import load_dotenv

import pandas as pd
import requests

In [2]:
load_dotenv()

True

## Configuration

In [4]:
url = 'https://api.github.com/graphql'
token = os.getenv('ICSC_GITHUB_ORG_PAT')

org_name = "ICSC-CN-HPC-Spoke-4-Earth-Climate"
proj_num = 3

## Get project ID

In [5]:
# Query GraphQL
query = """
query ($login: String!, $number: Int!) {
  organization(login: $login) {
    projectV2(number: $number) {
      id
    }
  }
}
"""

# Intestazioni della richiesta
headers = {
    'Authorization': f'Bearer {token}',
    "Accept": "application/vnd.github.v4+json",
    'Content-Type': 'application/json'
}

# Dati della richiesta (payload)
payload = {
    "query": query,
    "variables": {
        "login": org_name,
        "number": proj_num
    }
}

response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    print('Success:', response.json())
else:
    print('Error:', response.status_code, response.text)

Success: {'data': {'organization': {'projectV2': {'id': 'PVT_kwDOCY2ck84Aj7Zu'}}}}


In [6]:
project_id = response.json()['data']['organization']['projectV2']['id']
print(f'Project ID: {project_id}')

Project ID: PVT_kwDOCY2ck84Aj7Zu


## Get Table Fields IDs

In [7]:
query = f"""
query {{
  node(id: "{project_id}") {{
    ... on ProjectV2 {{
      fields(first: 20) {{
        nodes {{
          ... on ProjectV2Field {{
            id
            name
          }}
          ... on ProjectV2IterationField {{
            id
            name
            configuration {{
              iterations {{
                startDate
                id
              }}
            }}
          }}
          ... on ProjectV2SingleSelectField {{
            id
            name
            options {{
              id
              name
            }}
          }}
        }}
      }}
    }}
  }}
}}
"""

headers = {
    'Authorization': f'Bearer {token}',
    "Accept": "application/vnd.github.v4+json",
    'Content-Type': 'application/json'
}


data = {
    'query': query
}

response = requests.post(url, json=data, headers=headers)

if response.status_code == 200:
    print('Success:', response.json())
else:
    print('Error:', response.status_code, response.text)

Success: {'data': {'node': {'fields': {'nodes': [{'id': 'PVTF_lADOCY2ck84Aj7ZuzgcNuBQ', 'name': 'Title'}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcNuBU', 'name': 'Assignees'}, {'id': 'PVTSSF_lADOCY2ck84Aj7ZuzgcNuBY', 'name': 'Status', 'options': [{'id': 'f75ad846', 'name': 'Todo'}, {'id': '47fc9ee4', 'name': 'In Progress'}, {'id': '98236657', 'name': 'Done'}]}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcNuBc', 'name': 'Labels'}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcNuBg', 'name': 'Linked pull requests'}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcNuBk', 'name': 'Milestone'}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcNuBo', 'name': 'Repository'}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcNuB4', 'name': 'Reviewers'}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcdIVo', 'name': 'Repository name'}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcdIVs', 'name': 'WP'}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcdIWg', 'name': 'Task'}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcdIZs', 'name': 'Repository manager'}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcdIaw', 'name': 'Collaborators'}, {'id': 'PVTSSF_lADO

In [8]:
fields = response.json()['data']['node']['fields']['nodes']
#print(fields)
idx = next((index for (index,field) in enumerate(fields) if field['name'] == 'Repository name'),-1)
field_id = {field['name'] : field['id'] for field in fields[idx:]}
 
# For visibility (single-select field)
visib = [f for f in fields if f['name']=='Visibility'][0]
#print(visib)

visib_id = {}

visib_private = [f for f in visib['options'] if f['name'] == 'Private']
visib_id['Private'] = visib_private[0]['id']

visib_public = [f for f in visib['options'] if f['name'] == 'Public']
visib_id['Public'] = visib_public[0]['id']

#print
for n in field_id:
    print(f'{n}:\t {field_id[n]}')

print('\nVisibility Options:')
for i in visib_id:
    print(f'{i}:\t {visib_id[i]}')

Repository name:	 PVTF_lADOCY2ck84Aj7ZuzgcdIVo
WP:	 PVTF_lADOCY2ck84Aj7ZuzgcdIVs
Task:	 PVTF_lADOCY2ck84Aj7ZuzgcdIWg
Repository manager:	 PVTF_lADOCY2ck84Aj7ZuzgcdIZs
Collaborators:	 PVTF_lADOCY2ck84Aj7ZuzgcdIaw
Visibility:	 PVTSSF_lADOCY2ck84Aj7ZuzgcdIio

Visibility Options:
Private:	 8f5b6473
Public:	 169bd872


## Load the DataFrame

In [15]:
df_path = os.path.join(os.getcwd(),'Repository_list.csv')
df = pd.read_csv(df_path)
df.fillna("")
df

,Issue Number,Title,Node ID,Repository name,WP,Task,Repository manager,Collaborators,Private
0,15,[Repository Creation] - ML_clouds,I_kwDOMO4l_86Olge_,ML_clouds,3,3.3,fedef17,NaN,True
1,19,[Repository Creation] - OrogDrag,I_kwDOMO4l_86OloN6,OrogDrag,3,3.2,GuidoDavoli,NaN,True
2,20,[Repository Creation] - SurfAlb,I_kwDOMO4l_86Olo34,SurfAlb,3,3.1,francocatalano,NaN,True
3,21,[Repository Creation] - SoilHydraulics,I_kwDOMO4l_86OlpcD,SoilHydraulics,3,3.1,daniele-peano,NaN,True
4,22,[Repository Creation] - ML_entrainment,I_kwDOMO4l_86Olp-L,ML_entrainment,3,3.3,ValerioLembo,"ValerioLembo, turbulent-lab",True
5,23,[Repository Creation] - LandModelModules,I_kwDOMO4l_86OlrlM,LandModelModules,3,3.1,oxana-meteo,"GuidoDavoli, aalessan",True
6,24,[Repository Creation] - BEP_BEM_Offline,I_kwDOMO4l_86Olsdr,BEP_BEM_Offline,3,3.1,gpappac,NaN,True
7,25,[Repository Creation] - aero_drag_wrf,I_kwDOMO4l_86OltGQ,aero_drag_wrf,3,3.2,antoniolita,NaN,True
8,26,[Repository Creation] - EffCover,I_kwDOMO4l_86Olt7k,EffCover,3,3.1,emandica,NaN,True
9,27,[Repository Creation] - ShyConv,I_kwDOMO4l_86OlvT2,ShyConv,3,3.2,scausio,NaN,True


## Add items to the GitHub project

In [16]:
def add_item_query(project_id,issue_id):
  return f"""
  mutation {{
    addProjectV2ItemById(input: {{projectId: "{project_id}", contentId: "{issue_id}"}}) {{
      item {{
        id
      }}
    }}
  }}
  """



def update_field_query(project_id,item_id,field_id,value):

  #print('proj id: ',project_id)
  #print('item id: ',item_id)
  #print('field id: ',field_id)
  #print('value:',value)
  
  return f"""
  mutation {{
    updateProjectV2ItemFieldValue(input: {{
      projectId: "{project_id}",
      itemId: "{item_id}",  # Replace with the ID obtained from add_item_query
      fieldId: "{field_id}",
      value: {{
        text: "{value}"
      }}
    }}) {{
      projectV2Item {{
        id
      }}
    }}
  }}
  """



def set_single_select_field(project_id,item_id,field_id,option_id):

  #print('proj id: ',project_id)
  #print('item id: ',item_id)
  #print('field id: ',field_id)
  #print('option id:',option_id)
  
  return f"""
  mutation {{
    updateProjectV2ItemFieldValue(input: {{
      projectId: "{project_id}",
      itemId: "{item_id}",
      fieldId: "{field_id}",
      value: {{
        singleSelectOptionId: "{option_id}"
      }}
    }}) {{
      projectV2Item {{
        id
      }}
    }}
  }}
  """

In [19]:
def add_item(project_id,issue_id,field_id,visib_id):

    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }

    # Add new item to project
    response = requests.post(url, json={'query': add_item_query(project_id,issue_id)}, headers=headers)

    if response.status_code == 200:
        result = response.json()
        item_id = result['data']['addProjectV2ItemById']['item']['id']
        print(f'New item ID: {item_id}')

        # Update field of the new item
        for f in field_id:

            if f == 'Visibility':
            
                # Set visibility option ID
                if df.loc[df['Node ID'] == issue_id,'Private'].item() == False:
                    visib = 'Public'
                else:
                    visib = 'Private'

                option_id = visib_id[visib]
            
                # Update visibility field
                print(f'Updating {f} <- {visib}')
                response = requests.post(url, json={'query': set_single_select_field(project_id,item_id,field_id[f],option_id)}, headers=headers)

            else:
                value = df.loc[df["Node ID"] == issue_id][f].item()
                print(f'Updating {f} <- {value}')
                response = requests.post(url, json={'query': update_field_query(project_id,item_id,field_id[f],value)}, headers=headers)

            if response.status_code == 200:
                pass
                #result = response.json()     
                #updated_item_id = result['data']['updateProjectV2ItemFieldValue']['projectV2Item']['id']
                #print('Updated field ID:', updated_item_id)
            else:
                print('Error in field update:', response.status_code, response.text)
        
    else:
        print('\nErrore in item add:', response.status_code, response.text)
    print()
    
    return None

### Config and run

In [20]:
print(f'Project ID: {project_id}\n')

for iss_num in df['Issue Number']:
    issue_id = df.loc[df['Issue Number'] == iss_num]['Node ID'].item()
    print(f'Issue Number {iss_num} ID: {issue_id}\n')
    add_item(project_id,issue_id,field_id,visib_id)
    print(f'Issue {iss_num} added successfully!\n')

Project ID: PVT_kwDOCY2ck84Aj7Zu

Issue Number 15 ID: I_kwDOMO4l_86Olge_

New item ID: PVTI_lADOCY2ck84Aj7ZuzgQrJ00
Updating Repository name <- ML_clouds
Updating WP <- 3
Updating Task <- 3.3
Updating Repository manager <- fedef17
Updating Collaborators <- nan
Updating Visibility <- Private

Issue 15 added successfully!

Issue Number 19 ID: I_kwDOMO4l_86OloN6

New item ID: PVTI_lADOCY2ck84Aj7ZuzgQrK1A
Updating Repository name <- OrogDrag
Updating WP <- 3
Updating Task <- 3.2
Updating Repository manager <- GuidoDavoli
Updating Collaborators <- nan
Updating Visibility <- Private

Issue 19 added successfully!

Issue Number 20 ID: I_kwDOMO4l_86Olo34

New item ID: PVTI_lADOCY2ck84Aj7ZuzgQrK1M
Updating Repository name <- SurfAlb
Updating WP <- 3
Updating Task <- 3.1
Updating Repository manager <- francocatalano
Updating Collaborators <- nan
Updating Visibility <- Private

Issue 20 added successfully!

Issue Number 21 ID: I_kwDOMO4l_86OlpcD

New item ID: PVTI_lADOCY2ck84Aj7ZuzgQrK1s
Updating 